### <span style='color:Blue'> Fake News Detection.  </span> 
    Nesse projeto utilizei Jupyter, qualquer outra forma de "IDE" funcionaria. 
    
### <span style='color:Blue'> Introdução.  </span>
   ##### <span style='color:Blue'> Classificação.  </span> 
      No campo do machine learning, classificação refere-se a um tipo de modelo preditivo, o qual tenta prever a classe de um conjunto de dados. Um exemplo é o modelo de spam, o algoritmo analisa o conteúdo do email e decide o que ou não spam
***
   ##### <span style='color:Blue'> CountVectorizer.  </span>
       Vamos utilizar uma biblioteca do Sklearn chamada CountVectorizer. Vamos converter nosso texto em um dicionário que mapeia cada palavra única para o número de vezes que ela aparece nos dados.
 ***      
   #### <span style='color:Blue'> Naive Bayes.  </span> 
      A probabilidade de o evento A acontecer, dado o evento B acontecer.  O Teorema de Bayes calcula a probabilidade de que A seja verdadeiro dado o evento B com base na probabilidade inversa de B dado A. O método Naive Bayes para classificação de texto é muito popular porque pode ser escalado com muita facilidade. Naive Bayes assume independência condicional entre cada par de recursos, isso significa que estamos presumindo que as palavras em um artigo de notícias não têm impacto umas sobre as outras; estamos apenas examinando a probabilidade de ver cada palavra receber uma notícia falsa ou real
      

   
       

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [6]:
import pandas as pd
# Carregando os dois datasets
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [7]:
#Carregando os 'heads' dos datasets
fake.head()





,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
#Carregando os 'heads' dos datasets
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
# No head temos title, text, a subject, and date published
# Vamos flagear os conjuntos real = 0 fake = 1, coloquei uma coluna com 0 e 1 para a juntar os datasets 

In [10]:
true["fake_news"] = 0
fake["fake_news"] = 1

In [11]:
true.head(0)
fake.head(0)
#Criação de uma coluna 

,title,text,subject,date,fake_news


In [12]:

true.head()


,title,text,subject,date,fake_news
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


#### Overfitting 
    Todos os artigos começam com Reuters, se o modelo for treinado desta forma o modelo aprenderia que todos os inputs com Reuters seriam verdadeiros e os que nao tivesse essa marca seria falso, isso é chamado Overfitting  

In [13]:
just_text = true["text"]
just_text.head()

0    WASHINGTON (Reuters) - The head of a conservat...
1    WASHINGTON (Reuters) - Transgender people will...
2    WASHINGTON (Reuters) - The special counsel inv...
3    WASHINGTON (Reuters) - Trump campaign adviser ...
4    SEATTLE/WASHINGTON (Reuters) - President Donal...
Name: text, dtype: object

In [14]:

#Vou corrigir isso removendo todas as infos iniciais para tornar os textos dos dataframes parecidos. 
#Para fazer isso, o pandas tem uma função útil chamada extractall () que aceita um padrão de expressão regular (regex) como argumento. 
#Regex é uma sequência especial de caracteres que define um padrão de pesquisa. 
#Vamos extrair todo o texto que vem após o hífen que vem depois da Reuters.

In [15]:
just_text = just_text.str.extractall(r"^.*? - (?P<text>.*)")
#https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extractall.html

In [16]:
just_text = just_text.droplevel(1)
#https://www.w3resource.com/pandas/series/series-droplevel.php

In [17]:
true = true.assign(text=just_text["text"])

In [18]:
#Juntando os dois dataset

df = pd.concat([fake, true], axis = 0)

In [19]:
#Não vou utilizar essas colunas no treinamento 
df = df.drop(["subject", "date", "title"], axis = 1)

In [20]:
df.info()
df = df.dropna(axis = 0) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 21416
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       44869 non-null  object
 1   fake_news  44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [21]:
#Criaçao de um novo dataframe

clean_text = df.to_csv("cleaned_news.csv", index = False)

In [29]:
import pandas as pd
df = pd.read_csv("cleaned_news.csv")
df.head()



,text,fake_news
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [30]:
# Dividindo os Dados 
DV = "fake_news" # Variavel que mostra se é fake ou nao 
X = df.drop([DV], axis = 1)
y = df[DV]

In [31]:
from sklearn.model_selection import train_test_split
# treino em 75%
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

O CountVectorizer faz duas coisas. Primeiro, ele cria um pacote de palavras analisando todas as palavras exclusivas em todo o texto e mapeando-as para um dicionário onde o valor de cada palavra é o número de vezes que ela aparece. Em seguida, ele transforma o texto em uma matriz (um monte de vetores alinhados lado a lado)  É importante notar que não ajustamos o X_test porque queremos que nosso modelo realmente trabalhe com dados que ele não viu antes.

In [32]:

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_features = 10000)

X_train_counts = count_vect.fit_transform(X_train["text"])  
X_test = count_vect.transform(X_test["text"]) 

Antes de treinarmos nosso modelo, vamos falar sobre o que é especificamente o classificador Multinomial Bayes. Estamos calculando Naive Bayes em muitas palavras em todos os dados. Em nosso caso, isso significa responder à pergunta "Qual é a probabilidade de uma determinada palavra aparecer, visto que o artigo é verdadeiro?" Se você olhar para a fórmula do Teorema de Bayes, isso é em parte determinado pela probabilidade de que a notícia seja verdadeira (ou falsa!), Dado que existe uma palavra específica nela. Isso é repetido para cada palavra única em nosso vocabulário. O somatório de todos esses cálculos e a divisão por uma constante de normalização atribui o artigo a ser real ou não com base no resultado (uma probabilidade).


In [33]:
from sklearn.naive_bayes import MultinomialNB
Naive = MultinomialNB()
Naive.fit(X_train_counts, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
from sklearn.metrics import accuracy_score
# Vamos usar o modelo no conjunto de validaçao 
predictions_NB = Naive.predict(X_test)
# Use accuracy_score pra calcular a acuracia 
print("Accuracy Score:",accuracy_score(predictions_NB, y_test)*100) 

Accuracy Score: 94.32162595828133


Vamos testar 

In [35]:
#Site conhecido por noticias falsas 
onion = ["""Saying the social media network was the only place he had heard about his purported newborn relative, local man 
Eric Cruthirds expressed doubt Wednesday as to whether his cousin Audrey had actually given birth to the baby pictured in his Facebook feed. 
“The story seems plausible enough, but you always have to stop and consider the source, which in this case is an outlet that isn’t exactly known for its accuracy,” 
said Cruthirds, who observed that the photos his cousin shared of herself in a hospital bed ostensibly holding her child for the first time were a bit blurry and 
could have been photoshopped without too much effort. “Audrey was definitely pregnant at the family reunion,
but that doesn’t mean she carried the baby to term. Even if she did, there’s no guarantee the baby in these images is hers. 
Everyone’s liking and commenting on the post as if it were true, but let’s just say this wouldn’t be the first time my Aunt Brenda shared news on Facebook that turned out to be false. 
People really ought to check their facts before they spread around stuff like this.” At press time, Cruthirds was reportedly on the phone with the county clerk’s office, attempting to confirm whether there was any public record of the baby’s birth.
"""]

In [36]:
onion_vec = count_vect.transform(onion) #Vetor de palavras
predict_onion = Naive.predict(onion_vec) # Aplicando 
# 1: fake news!
print(predict_onion)

[1]


In [37]:
nyt = ["""Two top congressional Democrats opened an investigation on Saturday into President Trump’s removal of 
          Steve A. Linick, who led the office of the inspector general at the State Department, citing a pattern 
          of “politically-motivated firing of inspectors general.” Mr. Trump told Speaker Nancy Pelosi late 
          Friday night that he was ousting Mr. Linick, who was named by President Barack Obama to the State 
          Department post, and replacing him with an ambassador with close ties to Vice President Mike Pence in 
          the latest purge of inspectors general whom Mr. Trump has deemed insufficiently loyal to his 
          administration. In letters to the White House, State Department, and Mr. Linick, Representative Eliot 
          L. Engel of New York, the chairman of the House Foreign Affairs Committee, and Senator Bob Menendez of 
          New Jersey, the top Democrat on the Senate Foreign Relations Committee, requested that the administration
          turn over records and information related to the firing of Mr. Linick as well as “records of all I.G. 
          investigations involving the Office of the Secretary that were open, pending, or incomplete at the 
          time of Mr. Linick’s firing.” Mr. Engel and Mr. Menendez said in their letters that they believe 
          Secretary of State Mike Pompeo recommended Mr. Linick’s ouster because he had opened an investigation 
          into Mr. Pompeo’s conduct. The lawmakers did not provide any more details, but a Democratic aide said 
          that Mr. Linick had been looking into whether Mr. Pompeo had misused a political appointee at the State 
          Department to perform personal tasks for himself and his wife. “Such an action, transparently designed to
          protect Secretary Pompeo from personal accountability, would undermine the foundation of our democratic 
          institutions and may be an illegal act of retaliation,” the lawmakers wrote. Under law, the administration
          must notify Congress 30 days before formally terminating an inspector general. Mr. Linick is expected to 
          leave his post then. Mr. Trump’s decision to remove Mr. Linick is the latest in a series of ousters aimed
          at inspectors general who the president and his allies believe are opposed to his agenda. In May, Mr. 
          Trump moved to oust Christi A. Grimm, the principal deputy inspector general for the Department of Health
          and Human Services, whose office had issued a report revealing the dire state of the nation’s response to
          the pathogen. He has also taken steps to remove two other inspectors general, for the intelligence
          community and for the Defense Department. Mr. Linick was spotlighted during the impeachment inquiry when 
          he requested an urgent meeting with congressional staff members to give them copies of documents related 
          to the State Department and Ukraine, signaling they could be relevant to the House investigation into 
          whether President Trump pressured Ukraine to investigate former Vice President Joseph R. Biden Jr. and 
          his son Hunter Biden. The documents — a record of contacts between Rudolph W. Giuliani, the president’s 
          personal lawyer, and Ukrainian prosecutors, as well as accounts of Ukrainian law enforcement proceedings 
          — turned out to be largely inconsequential."""]

In [38]:
nyt_vec = count_vect.transform(nyt)
predict_nyt = Naive.predict(nyt_vec)
print(predict_nyt)

[0]
